# カメラの位置姿勢を求める

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skvideo.io import vread
import moviepy.editor as mpy
from tqdm import tqdm
from mpl_toolkits.mplot3d import axes3d, Axes3D
from IPython.display import Image

def npy_to_gif(npy, filename):
    clip = mpy.ImageSequenceClip(list(npy), fps=10)
    clip.write_gif(filename)

### 1. 素材

In [ ]:
vid = vread("src/shisa.mp4")
print(vid.shape)

### 2. アルコマーカー

In [ ]:
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco = cv2.aruco.drawMarker(aruco_dict, 0, 256)
plt.figure(figsize=(3,3)); plt.imshow(aruco); plt.show()
# cv2.imwrite("aruco.png", aruco)

### 3. カメラの設定

In [ ]:
marker_length = 0.07 # [m] ### 注意！
mtx = np.load("camera/mtx.npy")
dist = np.load("camera/dist.npy")
print(mtx); print(dist)

### 4. マーカーの検出

In [ ]:
frame = vid[0]
frame = frame[...,::-1]  # BGR2RGB
frame = cv2.resize(frame, (360, 640))

corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(frame, aruco_dict)
rvec, tvec, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_length, mtx, dist)

# ---- 描画
frame = cv2.aruco.drawDetectedMarkers(frame, corners, ids)
frame = cv2.aruco.drawAxis(frame, mtx, dist, rvec, tvec, marker_length/2)
# ----

plt.imshow(frame[...,::-1]); plt.show()

### 5. カメラの位置姿勢の計算

In [ ]:
# def eulerAnglesToRotationMatrix(euler):
#     R_x = np.array([[                1,                 0,                 0],
#                     [                0,  np.cos(euler[0]), -np.sin(euler[0])],
#                     [                0,  np.sin(euler[0]),  np.cos(euler[0])]])
#     R_y = np.array([[ np.cos(euler[1]),                 0,  np.sin(euler[1])],
#                     [ 0,                                1,                 0],
#                     [-np.sin(euler[1]),                 0,  np.cos(euler[1])]])
#     R_z = np.array([[ np.cos(euler[2]), -np.sin(euler[2]),                 0],
#                     [ np.sin(euler[2]),  np.cos(euler[2]),                 0],
#                     [                0,                 0,                 1]])
#     R = np.dot(R_z, np.dot(R_y, R_x))
#     return R

In [ ]:
XYZ = []
RPY = []
V_x = []
V_y = []
V_z = []

for frame in vid[:500:25]:  # 全部処理すると重いので…
    frame = frame[...,::-1]  # BGR2RGB
    frame = cv2.resize(frame, (360, 640))
    corners, ids, _ = cv2.aruco.detectMarkers(frame, aruco_dict)

    rvec, tvec, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_length, mtx, dist)

    R = cv2.Rodrigues(rvec)[0]  # 回転ベクトル -> 回転行列
    R_T = R.T
    T = tvec[0].T

    xyz = np.dot(R_T, - T).squeeze()
    XYZ.append(xyz)

    rpy = np.deg2rad(cv2.RQDecomp3x3(R_T)[0])
    RPY.append(rpy)
    # print(rpy)

    # rpy = cv2.decomposeProjectionMatrix(np.hstack([R_T, -T]))[6]  # [0~5]は使わない
    # rpy = np.deg2rad(rpy.squeeze())
    # print(rpy)

    # r = np.arctan2(-R_T[2][1], R_T[2][2])
    # p = np.arcsin(R_T[2][0])
    # y = np.arctan2(-R_T[1][0], R_T[0][0])
    # rpy = - np.array([r, p, y])
    # print(rpy)

    # from scipy.spatial.transform import Rotation
    # diff = eulerAnglesToRotationMatrix(rpy) - R_T
    # print(diff.astype(np.float16))
    # diff = Rotation.from_euler('xyz', rpy).as_matrix() - R_T
    # print(diff.astype(np.float16))
    
    V_x.append(np.dot(R_T, np.array([1,0,0])))
    V_y.append(np.dot(R_T, np.array([0,1,0])))
    V_z.append(np.dot(R_T, np.array([0,0,1])))

    # ---- 描画
    # cv2.aruco.drawDetectedMarkers(frame, corners, ids, (0,255,255))
    # cv2.aruco.drawAxis(frame, mtx, dist, rvec, tvec, marker_length/2)
    # cv2.imshow('frame', frame)
    # cv2.waitKey(1)
    # ----

cv2.destroyAllWindows()

In [ ]:
def plot_all_frames(elev=90, azim=270):
    frames = []

    for t in tqdm(range(len(XYZ))):
        fig = plt.figure(figsize=(4,3))
        ax = Axes3D(fig)
        ax.view_init(elev=elev, azim=azim)
        ax.set_xlim(-2, 2); ax.set_ylim(-2, 2); ax.set_zlim(-2, 2)
        ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")

        x, y, z = XYZ[t]
        ux, vx, wx = V_x[t]
        uy, vy, wy = V_y[t]
        uz, vz, wz = V_z[t]

        # draw marker
        ax.scatter(0, 0, 0, color="k")
        ax.quiver(0, 0, 0, 1, 0, 0, length=1, color="r")
        ax.quiver(0, 0, 0, 0, 1, 0, length=1, color="g")
        ax.quiver(0, 0, 0, 0, 0, 1, length=1, color="b")
        ax.plot([-1,1,1,-1,-1], [-1,-1,1,1,-1], [0,0,0,0,0], color="k", linestyle=":")

        # draw camera
        if t < 5:
            ax.quiver(x, y, z, ux, vx, wx, length=0.5, color="k")
            ax.quiver(x, y, z, uy, vy, wy, length=0.5, color="k")
            ax.quiver(x, y, z, uz, vz, wz, length=0.5, color="k")
        else:
            ax.quiver(x, y, z, ux, vx, wx, length=0.5, color="r")
            ax.quiver(x, y, z, uy, vy, wy, length=0.5, color="g")
            ax.quiver(x, y, z, uz, vz, wz, length=0.5, color="b")

        # save for animation
        fig.canvas.draw()
        frames.append(np.array(fig.canvas.renderer.buffer_rgba()))
        plt.close()

    return frames

In [ ]:
frames = plot_all_frames(elev=105, azim=270)
npy_to_gif(frames, "src/sample1.gif"); Image(url='src/sample1.gif')

In [ ]:
frames = plot_all_frames(elev=165, azim=270)
npy_to_gif(frames, "src/sample2.gif"); Image(url='src/sample2.gif')

In [ ]:
plt.title("xyz"); plt.plot(XYZ); plt.show()  # 青:x, 橙:y, 緑:z
plt.title("rpy"); plt.plot(RPY); plt.show()  # 青:r, 橙:p, 緑:y
plt.title("(v_x)"); plt.plot(V_x); plt.show()
plt.title("(v_y)"); plt.plot(V_y); plt.show()
plt.title("(v_z)"); plt.plot(V_z); plt.show()